# Evaluation

To evaluate our database's performance, we analyze every operator separately and perform an integration test afterwards.

In [1]:
import pandas as pd
import requests
from sklearn.metrics import classification_report

from models import ModelMgr
from db.criteria import *
from db.operators import *
from db.operators.Aggregate import *
from models.text_generation.LLaMA import LLaMATextGenerationModel

from models.semantic_validation import *
from models.embedding import *

from db.db import DBConnector

In [2]:
# Load Models
m = ModelMgr()
em = SentenceTransformerEmbeddingModel(m)
sv = LLaMAValidationModel(m)
gm = LLaMATextGenerationModel(m)

# Load DB
db = DBConnector("../config.ini", load_db=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Evaluate Select Operator

### Company Data

We used the [People Data Labs 2019 Global Company Dataset](https://www.kaggle.com/datasets/peopledatalabssf/free-7-million-company-dataset) to evaluate the `SoftEqual`- and `SoftValidate`-Operator.

#### Data Understanding

First, we evaluate the data understanding. The dataset contains a size estimate as ranges ('1 - 10', '5001 - 10000', '10001+').
We test if the `SoftValidate` is able to determine if an arbitrary test number is 1) bigger than, 2) smaller than or 3) in between a given size range.

So, we test the performance of:
* $\sigma_{\checkmark ( \text{ Is x in range \{size\_range\}? })}(Companies)$
* $\sigma_{\checkmark ( \text{ Is x lower than the range \{size\_range\}? })}(Companies)$
* $\sigma_{\checkmark ( \text{ Is x bigger than the range \{size\_range\}? })}(Companies)$

In [3]:
# Determine all size ranges
scan_companies = Scan("companies", em=em, sv=sv, db=db)
agg = HashAggregate(scan_companies, ["size_range"], [CountAggregation("size_range", "size_range_count")])
size_ranges = {row["size_range"] for row in agg}
size_ranges

{'1 - 10',
 '10001+',
 '1001 - 5000',
 '11 - 50',
 '201 - 500',
 '5001 - 10000',
 '501 - 1000',
 '51 - 200'}

In [4]:
gt =  [
    (8, "=", "1 - 10"), (-5, "<", "1 - 10"), (35, ">", "1 - 10"),
    (35, "=", "11 - 50"), (8, "<", "11 - 50"), (172, ">", "11 - 50"),
    (172, "=", "51 - 200"), (35, "<", "51 - 200"), (354, ">", "51 - 200"),
    (354, "=", "201 - 500"), (172, "<", "201 - 500"), (502, ">", "201 - 500"),
    (502, "=", "501 - 1000"), (354, "<", "501 - 1000"), (3581, ">", "501 - 1000"),
    (3581, "=", "1001 - 5000"), (502, "<", "1001 - 5000"), (6000, ">", "1001 - 5000"),
    (6000, "=", "5001 - 10000"), (3581, "<", "5001 - 10000"), (32100, ">", "5001 - 10000"),
    (32100, "=", "10001+"), (6000, "<", "10001+"),
]


results = []
for row in gt:
    result = {"test_no": row[0], "gt": row[1], "size_range": row[2]}
    data = Dummy("data", ["test_no", "size_range"], [(row[0], row[2])])
    sel_equal = Select(data, SoftValidate("Is {test_no} in range of {size_range}?", sv=sv, full_record=False))
    result["="] = len(list(sel_equal)) > 0
    sel_greater = Select(data, SoftValidate("Is {test_no} lower than the range {size_range}?", sv=sv, full_record=False))
    result[">"] = len(list(sel_greater)) > 0
    sel_lower = Select(data, SoftValidate("Is {test_no} bigger than the range {size_range}", sv=sv, full_record=False))
    result["<"] = len(list(sel_lower)) > 0
    results.append(result)

df_eval = pd.DataFrame(results)
df_eval.head()

,test_no,gt,size_range,=,>,<
0,8,=,1 - 10,False,False,False
1,-5,<,1 - 10,False,False,False
2,35,>,1 - 10,False,False,False
3,35,=,11 - 50,True,False,False
4,8,<,11 - 50,True,True,False


In [5]:
print("Classification for {test_no} in range of {size_range}")
print(classification_report(df_eval["gt"] == "=", df_eval["="]))

Classification for {test_no} in range of {size_range}
              precision    recall  f1-score   support

       False       0.71      0.33      0.45        15
        True       0.38      0.75      0.50         8

    accuracy                           0.48        23
   macro avg       0.54      0.54      0.48        23
weighted avg       0.60      0.48      0.47        23



In [6]:
print("Classification for {test_no} lower than range {size_range}")
print(classification_report(df_eval["gt"] == ">", df_eval[">"]))

Classification for {test_no} lower than range {size_range}
              precision    recall  f1-score   support

       False       0.71      0.31      0.43        16
        True       0.31      0.71      0.43         7

    accuracy                           0.43        23
   macro avg       0.51      0.51      0.43        23
weighted avg       0.59      0.43      0.43        23



In [7]:
print("Classification for {test_no} bigger than range {size_range}")
print(classification_report(df_eval["gt"] == "<", df_eval["<"]))

Classification for {test_no} bigger than range {size_range}
              precision    recall  f1-score   support

       False       0.56      0.60      0.58        15
        True       0.14      0.12      0.13         8

    accuracy                           0.43        23
   macro avg       0.35      0.36      0.36        23
weighted avg       0.42      0.43      0.43        23



#### Domain Knowledge

To evaluate the domain knowledge of the operators (and the LLM), we predict if a company has 'automotive' as industry.
Therefore, we collect all big companies (`size > 1000`) as ground truth and store the classification value (true for all rows with 'automotive'=industry).

To generate the predictions, we execute two other queries and classify the row with `True` if the query returns a result. To avoid data leakage, we remove the 'industry' column first.
 * `SoftEqual`-Operator: $ \sigma_{\text{(name, country, size\_range, locality, year\_founded}) \approx \text{'car company'}} (Companies)$
 * `SoftValidate`-Operator: $\sigma_{\checkmark ( \text{Is this company record:  \{name: \{name\}, country:\{country\}, size\_range: \{size\_range\}, locality: \{locality\}, year\_founded: \{year\_founded\}\} of a car company? })}(Companies)$

In [8]:
scan_companies = Scan("companies", em=em, sv=sv, db=db, sql_annex=f"  WHERE size_range IN ('10001+', '1001 - 5000')")
gt = [row for row in scan_companies]
len(gt), gt[:3]

(26632,
 [RealDictRow([('id', 4784855),
               ('name', 'td'),
               ('domain', 'td.com'),
               ('year_founded', 1955),
               ('industry', 'banking'),
               ('size_range', '10001+'),
               ('locality', 'toronto, ontario, canada'),
               ('country', 'canada'),
               ('linkedin_url', 'linkedin.com/company/td'),
               ('current_employee_estimate', '42136'),
               ('total_employee_estimate', '93645')]),
  RealDictRow([('id', 6770635),
               ('name', 'hrt'),
               ('domain', None),
               ('year_founded', None),
               ('industry', 'broadcast media'),
               ('size_range', '1001 - 5000'),
               ('locality', None),
               ('country', None),
               ('linkedin_url', 'linkedin.com/company/hrt'),
               ('current_employee_estimate', '793'),
               ('total_employee_estimate', '1277')]),
  RealDictRow([('id', 3383694),
        

In [9]:
p = Project(scan_companies, [x.column_name for x in scan_companies.table.table_structure if x.column_name != 'industry'], em=em)
sel = Select(p, SoftEqual(["name", "country", "size_range", "locality", "year_founded"], Constant('car company'), em=em, threshold=0.3))
str(sel)

'σ_{name, country, size_range, locality, year_founded ≈ \'car company\'} (π_{"country"≈>country, "current_employee_estimate"≈>current_employee_estimate, "domain"≈>domain, "id"≈>id, "linkedin_url"≈>linkedin_url, "locality"≈>locality, "name"≈>name, "size_range"≈>size_range, "total_employee_estimate"≈>total_employee_estimate, "year_founded"≈>year_founded} ("companies"≈>"people_data_labs.companies"))'

In [10]:
prediction = [row for row in sel]
len(prediction), prediction[0:3]

(3056,
 [{'country': 'united states',
   'current_employee_estimate': '122031',
   'domain': 'cognizant.com',
   'id': 2959148,
   'linkedin_url': 'linkedin.com/company/cognizant',
   'locality': 'teaneck, new jersey, united states',
   'name': 'cognizant technology solutions',
   'size_range': '10001+',
   'total_employee_estimate': '210020',
   'year_founded': 1994},
  {'country': 'united states',
   'current_employee_estimate': '115188',
   'domain': 'att.com',
   'id': 3300741,
   'linkedin_url': 'linkedin.com/company/at&t',
   'locality': 'dallas, texas, united states',
   'name': 'at&t',
   'size_range': '10001+',
   'total_employee_estimate': '269659',
   'year_founded': 1876},
  {'country': 'united kingdom',
   'current_employee_estimate': '66632',
   'domain': 'vodafone.com',
   'id': 1379366,
   'linkedin_url': 'linkedin.com/company/vodafone',
   'locality': 'berks, west berkshire, united kingdom',
   'name': 'vodafone',
   'size_range': '10001+',
   'total_employee_estimate'

In [11]:
df_gt = pd.DataFrame(gt).set_index('id')
df_prediction = pd.DataFrame(prediction).set_index('id')
df_merge = df_gt.merge(df_prediction, left_index=True, right_index=True, how='left')
df_merge["class"] = df_merge["industry"] == "automotive"
df_merge["prediction"] = df_merge["name_y"].notna()

print(classification_report(df_merge["class"], df_merge["prediction"]))

              precision    recall  f1-score   support

       False       0.99      0.89      0.94     26112
        True       0.10      0.59      0.17       520

    accuracy                           0.89     26632
   macro avg       0.55      0.74      0.56     26632
weighted avg       0.97      0.89      0.93     26632



In [12]:
p = Project(scan_companies, [x.column_name for x in scan_companies.table.table_structure if x.column_name != 'industry'], em=em)
sel = Select(p, SoftValidate("Is this company record: {{name: {name}, country: {country}, size_range: {size_range}, locality: {locality}, year_founded: {year_founded}}} of a car company? ", sv=sv, full_record=False))
str(sel)

'σ_{✓_{Is this company record: {{name: {name}, country: {country}, size_range: {size_range}, locality: {locality}, year_founded: {year_founded}}} of a car company? }} (π_{"country"≈>country, "current_employee_estimate"≈>current_employee_estimate, "domain"≈>domain, "id"≈>id, "linkedin_url"≈>linkedin_url, "locality"≈>locality, "name"≈>name, "size_range"≈>size_range, "total_employee_estimate"≈>total_employee_estimate, "year_founded"≈>year_founded} ("companies"≈>"people_data_labs.companies"))'

In [13]:
prediction = [row for row in sel]
len(prediction), prediction[0:3]

(101,
 [{'country': 'united states',
   'current_employee_estimate': '2132',
   'domain': 'vw.com',
   'id': 4967841,
   'linkedin_url': 'linkedin.com/company/volkswagen-of-america-inc',
   'locality': 'herndon, virginia, united states',
   'name': 'volkswagen of america, inc',
   'size_range': '1001 - 5000',
   'total_employee_estimate': '4409',
   'year_founded': 1955},
  {'country': 'united kingdom',
   'current_employee_estimate': '1865',
   'domain': 'bentleymotors.com',
   'id': 5163300,
   'linkedin_url': 'linkedin.com/company/bentley-motors-ltd',
   'locality': 'crewe, cheshire east, united kingdom',
   'name': 'bentley motors ltd',
   'size_range': '1001 - 5000',
   'total_employee_estimate': '4142',
   'year_founded': 1919},
  {'country': 'united states',
   'current_employee_estimate': '1753',
   'domain': 'hondaresearch.com',
   'id': 3260108,
   'linkedin_url': 'linkedin.com/company/honda-r&d',
   'locality': 'raymond, ohio, united states',
   'name': 'honda r&d americas, 

In [14]:
df_gt = pd.DataFrame(gt).set_index('id')
df_prediction = pd.DataFrame(prediction).set_index('id')
df_merge = df_gt.merge(df_prediction, left_index=True, right_index=True, how='left')
df_merge["class"] = df_merge["industry"] == "automotive"
df_merge["prediction"] = df_merge["name_y"].notna()

print(classification_report(df_merge["class"], df_merge["prediction"]))

              precision    recall  f1-score   support

       False       0.98      1.00      0.99     26112
        True       0.82      0.16      0.27       520

    accuracy                           0.98     26632
   macro avg       0.90      0.58      0.63     26632
weighted avg       0.98      0.98      0.98     26632



## Evaluate SoftJoin Operator

To evaluate the soft Join Operator, we use the test cases from the [Semantic-Join GitHub Repository](https://github.com/Yeye-He/Semantic-Join).
Each test case is present as two relations with one column each, and the matching records as ground truth.

E.g. Country->Continent
    A: \["Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Luanda", ...\]
    B: \["Rome", "Amsterdam", "Reykjavik", "Bridgetown", ...\]
    GT: \["Afghanistan\tKabul", "Albania\tTirana", "Algeria\tAlgiers", "Andorra\tAndorra la Vella", ...\]

In [15]:
def convert_to_operators(case: int):
    case_url = f"https://raw.githubusercontent.com/Yeye-He/Semantic-Join/refs/heads/master/Semantic-join-Benchmark/Case{case}_input.txt"
    gt_url = f"https://raw.githubusercontent.com/Yeye-He/Semantic-Join/refs/heads/master/Semantic-join-Benchmark/Case{case}_groundtruth.txt"
    input_data = requests.get(case_url).text
    gt_data = requests.get(gt_url).text
    d1, d2 = input_data.split("\n\n")
    d1, d2 = [(x, ) for x in d1.split("\n")], [(x, ) for x in d2.split("\n")]
    n = len(d1) * len(d2)
    gt = {x for x in gt_data.split("\n") if x.strip() != ""}
    return (Dummy("A", ["a"], d1), Dummy("B", ["b"], d2)), gt, n

def calculate_metrics(operator: Operator, gt: set, n):
    joined_values = {f'{row["a"]}\t{row["b"]}' for row in operator}

    tps, fns, fps = gt & joined_values, gt - joined_values, joined_values - gt
    tp, fn, fp = len(tps), len(fns), len(fps)
    tn = n - tp - fn - fp

    values = {"tp": tp, "fn": fn, "fp": fp, "tn": tn}
    results = {"tps": tps, "fns": fns, "fps": fps}

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    accuracy = (tp + tn) / (tp + fp + tn + fn) if (tp + fp + tn + fn) > 0 else 0

    scores = {
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1_score,
        "Specificity": specificity,
        "Accuracy": accuracy
    }

    return values, scores, results

In [16]:
semantic_join_test_cases = [
    (1, "Is {a} in {b}?", "Country->Continent", 0.5),
    # (9, "Is {a} in {b}?", "University->US State"),
    # (10, "Is {b} the capital of {a}?", "Countries->Capital Cities", 0.5),
    #(18, "{a} has headquarter in {b}?", "Companies->Headquarter"),
    #(19, "{b} is or was CEO of {a}?", "CEO<-Companies")

]

total_scores = []
total_values = []
for case_no, sv_template, descr, threshold in semantic_join_test_cases:
    print(f"Evaluating case {case_no} ({descr}):", end=" ")
    (table_a, table_b), gt_countries_continent, n_options = convert_to_operators(case_no)
    sij = InnerSoftJoin(table_a, table_b, Column("a"), Column("b"), em=em, use_semantic_validation=True, sv=sv, sv_template=sv_template, threshold=threshold)
    v, s, r = calculate_metrics(sij, gt_countries_continent, n_options)
    total_values.append(v)
    total_scores.append(s)
    print(f"\t{s}")

Evaluating case 1 (Country->Continent): 	{'Precision': 0.9166666666666666, 'Recall': 0.7938144329896907, 'F1 Score': 0.850828729281768, 'Specificity': 0.9879725085910653, 'Accuracy': 0.9602356406480118}
